## Data Cleaning & Extraction

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# read files 
jan = pd.read_csv("Resources/201901-citibike-tripdata.csv")
feb = pd.read_csv("Resources/201902-citibike-tripdata.csv")
mar = pd.read_csv("Resources/201903-citibike-tripdata.csv")
apr = pd.read_csv("Resources/201904-citibike-tripdata.csv")
may = pd.read_csv("Resources/201905-citibike-tripdata.csv")
jun = pd.read_csv("Resources/201906-citibike-tripdata.csv")
jul = pd.read_csv("Resources/201907-citibike-tripdata.csv")
aug = pd.read_csv("Resources/201908-citibike-tripdata.csv")
sep = pd.read_csv("Resources/201909-citibike-tripdata.csv")
oct = pd.read_csv("Resources/201910-citibike-tripdata.csv")
nov = pd.read_csv("Resources/201911-citibike-tripdata.csv")
dec = pd.read_csv("Resources/201912-citibike-tripdata.csv")

Check whether we need to remove any records,and it shows that only start & end stations are missing some records.
I keep it as it is not wrong records but just missing information, and it would not impact results of station analysis as
it is just a small scale.


In [ ]:
df_list = [jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec]
for df in df_list:
    print(f"There are {df.shape[0]} row")
    print(df.isna().sum())
    print("="*40)

In [ ]:
# Filter the columns that we want & add Month column
drop_columns = ["stoptime","end station id","end station latitude","end station longitude","bikeid"]

for df in df_list: 
    df.drop(columns = drop_columns, inplace=True)
    df["month"]= [time[5:7]for time in df["starttime"]]

# Make the final data for analysis
ana_df = pd.concat(df_list, axis=0, ignore_index=True)
ana_df.head()

In [ ]:
# Save it for further analysis
ana_df.to_csv("Resources/2019_ana.csv")

## Analysis 1:　Ｍonthly total trips (by customer type)

In [ ]:
# ana_df["usertype"].unique()
mon_total = ana_df[["usertype", "month"]]
x = mon_total["month"].unique()
y_sub = mon_total.loc[mon_total["usertype"]=="Subscriber"].groupby("month").count()["usertype"]/1000000
y_cus = mon_total.loc[mon_total["usertype"]=="Customer"].groupby("month").count()["usertype"]/1000000

In [ ]:
plt.figure(figsize=[10,6])
plt.plot(x, y_sub, "o-c", label="Subscriber")
plt.plot(x, y_cus, "o-g", label="Customer")
plt.legend(loc="upper left")
plt.ylim(0, max(y_sub+0.2))
plt.title("Monthly Total Trips", fontsize=20)
plt.ylabel("Total Trips (MM)", fontsize=15)
plt.savefig("analysis/1.jpg")
plt.show()

## Analysis 2: Top Age Groups Utilising Program

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

ana_df = pd.read_csv("Resources/2019_ana.csv")

In [ ]:
from datetime import datetime
age_group = ana_df[["usertype", "birth year"]]
age_group["age"]= datetime.now().year - age_group["birth year"]

In [ ]:
print(age_group["age"].min(), age_group["age"].max())
print(age_group["age"].value_counts().to_string())

Age groups would be categorised as:
under 20, 20-30, 30-40, 40-50, 50-60,60-70,70-80, older than 80

In [ ]:
bins = [0,20,30,40,50,60,70,80,200]
labels = ["under 20","20-30","30-40","40-50","50-60","60-70","70-80", "Other"]
age_group["age_group"]=pd.cut(x=age_group['age'],bins=bins, labels=labels)
age_group.head()

In [ ]:
# get plotting data
y_sub = age_group.loc[age_group["usertype"]=="Subscriber"].groupby("age_group").count()["usertype"] / 1000
y_cus = age_group.loc[age_group["usertype"]=="Customer"].groupby("age_group").count()["usertype"] / 1000
base = age_group.groupby("age_group").count()["usertype"]
per = y_sub*1000/base
label = [str(per[i]*100)[:4]+"%" for i in range(len(per))]

In [ ]:
plt.figure(figsize=[10,8])
plt.bar(labels, y_sub,color="c", label="Subscriber (with % of respective age groups at the top)")
plt.bar(labels, y_cus, bottom=y_sub, color="g", label="Customer")
plt.title("Total Trips by Age Groups", fontsize=20)
plt.ylabel("Trip No. (K)", fontsize=15)
plt.legend(loc="upper right")
for i in range(len(label)):
    plt.text(labels[i], (y_sub[i]+y_cus[i]+100), label[i], ha="center")
plt.savefig("analysis/2.jpg")
plt.show()

## Analysis 3: Total Trips by Gender

In [ ]:
ana_df.head()

In [ ]:
gender_df = ana_df[["usertype", "gender"]]
x_label=["Unknown", "Male","Female"]
y_sub = gender_df.loc[gender_df["usertype"]=="Subscriber"].groupby("gender").count()["usertype"] / 1000
y_cus = gender_df.loc[gender_df["usertype"]=="Customer"].groupby("gender").count()["usertype"] / 1000
base = gender_df.groupby("gender").count()["usertype"]
per = y_sub*1000/base
label = [str(per[i]*100)[:4]+"%" for i in range(len(per))]

In [ ]:
plt.figure(figsize=[10,5])
plt.barh(x_label, y_sub,color="c", label="Subscriber (with % of respective age groups at the right)")
plt.barh(x_label, y_cus, left=y_sub, color="g", label="Customer")
plt.title("Total Trips by Gender", fontsize=20)
plt.xlabel("Trip No. (K)", fontsize=15)
plt.legend(loc="upper right")
for i in range(len(label)):
    plt.text((y_sub[i]+y_cus[i]+450),x_label[i],label[i], ha="center")
plt.savefig("analysis/3.jpg")
plt.show()

## Analysis 4: Peak Hours In A Day


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

ana_df = pd.read_csv("Resources/2019_ana.csv")
ana_df.head()

In [ ]:
from datetime import datetime
time_table = ana_df[["starttime", "birth year"]]
time_table["time"] = [time_table["starttime"][i][11:13] for i in range(time_table.shape[0])]
time_table["age"] = datetime.now().year - time_table["birth year"]
bins = [0,20,30,40,50,60,70,80,200]
labels = ["under 20","20-30","30-40","40-50","50-60","60-70","70-80", "Other"]
time_table["age_group"]=pd.cut(x=time_table['age'],bins=bins, labels=labels)
time_table.head()

In [ ]:
x = time_table["time"].unique()
y = []

for group in labels:
    y.append(time_table.loc[time_table["age_group"] == group].groupby("time").count()["age_group"]/1000)

In [ ]:
plt.figure(figsize=[12,8])
plt.stackplot(x,y,labels=labels, alpha=0.8)
plt.title("Hourly Total Trips", fontsize=20)
plt.ylabel("Trip No. (K)", fontsize=15)
plt.xlim([0,23])
plt.legend(loc="upper left")
plt.savefig("analysis/4.jpg")
plt.show()

## Analysis 5: Start Stations Map

understand trip durations
* quantitibely determine whether outliers
* draw outliers in the box plot
* how to deal with outliers
* draw the map